# LongContextRAG 使用范例

本 notebook 展示了如何使用 `LongContextRAG` 类来处理长文本上下文的检索增强生成（RAG）任务。

In [1]:
import os
import byzerllm
from autocoder.common import AutoCoderArgs
from autocoder.rag.long_context_rag import LongContextRAG

# 初始化 ByzerLLM
llm = byzerllm.ByzerLLM.from_default_model("deepseek_chat")

# 设置参数
args = AutoCoderArgs()

# 指定包含文档的目录路径
# /Users/allwefantasy/projects/llm_friendly_packages/github.com/allwefantasy/byzer-sql
docs_path = "~/workspace/dbt/demo"  # 请替换为实际的文档目录路径

# 初始化 LongContextRAG
long_context_rag = LongContextRAG(llm, args, docs_path)

2024-08-27 13:29:19,024	INFO worker.py:1586 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-08-27 13:29:19,029	INFO worker.py:1762 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


(raylet) The autoscaler failed with the following error:
Terminated with signal 15
  File "/Users/jayce.zhao/miniconda3/envs/python310/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 709, in <module>
    monitor.run()
  File "/Users/jayce.zhao/miniconda3/envs/python310/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 584, in run
    self._run()
  File "/Users/jayce.zhao/miniconda3/envs/python310/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 438, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)



## 使用 LongContextRAG 进行问答

In [3]:
# 定义一个问题
question = "ads_weekly_report_1有多少个字段?"

# 使用 LongContextRAG 进行问答
response_generator, contexts = long_context_rag.stream_chat_oai(
    conversations=[{"role": "user", "content": question}]
)

# 打印回答
print("问题:", question)
print("回答:")
for chunk in response_generator:
    print(chunk, end="")
print("\n")

# 打印相关上下文
print("相关文档:")
for context in contexts:
    print(f"- {context}")

问题: ads_weekly_report_1有多少个字段?
回答:
没有找到相关的文档来回答这个问题。

相关文档:


## 多轮对话示例

In [ ]:
# 定义多轮对话
conversations = [
    {"role": "user", "content": "什么是深度学习？"},
    {"role": "assistant", "content": "深度学习是机器学习的一个子领域，它使用多层神经网络来模拟人脑的学习过程。深度学习模型可以自动从大量数据中学习特征，无需人工特征工程。这种方法在图像识别、自然语言处理和语音识别等领域取得了巨大突破。"},
    {"role": "user", "content": "深度学习和传统机器学习有什么区别？"}
]

# 使用 LongContextRAG 进行多轮对话
response_generator, contexts = long_context_rag.stream_chat_oai(conversations)

# 打印回答
print("问题: 深度学习和传统机器学习有什么区别？")
print("回答:")
for chunk in response_generator:
    print(chunk, end="")
print("\n")

# 打印相关上下文
print("相关文档:")
for context in contexts:
    print(f"- {context}")

## 解释 LongContextRAG 的工作原理

LongContextRAG 的工作原理如下：

1. 文档加载：从指定目录加载所有的 Markdown 文档。

2. 相关性检查：对于给定的问题，使用 LLM 并行检查每个文档的相关性。

3. 答案生成：使用相关文档作为上下文，生成问题的答案。

4. 流式输出：以流式方式返回生成的答案，同时返回相关文档的信息。

这种方法允许处理大量文档，同时保持较低的延迟，因为它只在相关文档上进行深入处理。